## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")



In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x119f38290>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x119f38c20>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, nice to meet you! It's fascinating to connect with a Chief AI Engineer. \n\nWhat are you currently working on that you find most exciting? I'm always eager to learn about the latest advancements in AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 22, 'total_tokens': 73, 'completion_time': 0.092727273, 'prompt_time': 0.00014055, 'queue_time': 0.020029747, 'total_time': 0.092867823}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b31a0995-8c8c-4f89-9cfb-bee896359b9f-0', usage_metadata={'input_tokens': 22, 'output_tokens': 51, 'total_tokens': 73})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Krish, and that you are a Chief AI Engineer!  😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm happy to chat. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 97, 'total_tokens': 147, 'completion_time': 0.090909091, 'prompt_time': 0.00343392, 'queue_time': 0.021065378, 'total_time': 0.094343011}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5dd15fc-d21b-4b1d-9373-98edabeaba98-0', usage_metadata={'input_tokens': 97, 'output_tokens': 50, 'total_tokens': 147})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [10]:
response.content

"Hello Krish, it's nice to meet you! \n\nAs a Chief AI Engineer, I imagine you're involved in some exciting projects. What are some of the things you're working on these days? \n\nI'm always eager to learn more about the applications of AI and how it's being used to make a difference.\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  \n\nYou told me at the beginning of our conversation! 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 110, 'total_tokens': 132, 'completion_time': 0.04, 'prompt_time': 0.003650149, 'queue_time': 0.02116333, 'total_time': 0.043650149}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-418783d8-c2f9-4497-a291-9c75d9f25344-0', usage_metadata={'input_tokens': 110, 'output_tokens': 22, 'total_tokens': 132})

In [12]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today? 😊  \n\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.  \n\nI remember! 😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish! It's nice to meet you.\n\nI'm happy to help answer any questions you have to the best of my ability. What can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_time': 0.078181818, 'prompt_time': 0.000572968, 'queue_time': 0.019425999, 'total_time': 0.078754786}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae44d84a-3b32-4697-9917-7f78e6cc60de-0', usage_metadata={'input_tokens': 31, 'output_tokens': 43, 'total_tokens': 74})

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hello Krish! It's nice to meet you.\n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\nWhat can I do for you today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75, 'completion_time': 0.08, 'prompt_time': 0.000308829, 'queue_time': 0.020810469, 'total_time': 0.080308829}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-21ee0424-d01c-48ea-869c-6023305e5a44-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!  😄  I remember. \n\n\n\n'

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी क्षमता से उत्तर देने की कोशिश करूँगा। \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [22]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते, कृष्णा! 👋  क्या मैं तुमकी मदद कर सकता हूँ? 😊 \n'

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [31]:
response.content

'आपका नाम कृष है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [37]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"